In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reduced-http/http.csv
/kaggle/input/answers/r6.1-5.csv
/kaggle/input/answers/r6.1-3.csv
/kaggle/input/answers/r5.1-1.csv
/kaggle/input/answers/r3.1-1.csv
/kaggle/input/answers/r3.2-1.csv
/kaggle/input/answers/r4.1-2.csv
/kaggle/input/answers/r6.2-3.csv
/kaggle/input/answers/r6.2-5.csv
/kaggle/input/answers/r5.1-2.csv
/kaggle/input/answers/r6.1-1.csv
/kaggle/input/answers/readme.txt
/kaggle/input/answers/r6.2-4.csv
/kaggle/input/answers/r4.1-3.csv
/kaggle/input/answers/scenarios.txt
/kaggle/input/answers/r5.1-3.csv
/kaggle/input/answers/license.txt
/kaggle/input/answers/insiders.csv
/kaggle/input/answers/r3.2-2.csv
/kaggle/input/answers/r6.1-2.csv
/kaggle/input/answers/r4.1-1.csv
/kaggle/input/answers/r2.csv
/kaggle/input/answers/r3.1-2.csv
/kaggle/input/answers/r6.2-2.csv
/kaggle/input/answers/r5.1-4.csv
/kaggle/input/answers/r6.2-1.csv
/kaggle/input/cert-insider-threat-detection-research/users.csv
/kaggle/input/cert-insider-threat-detection-research/logon.csv
/kaggle/inp

In [2]:
import pandas as pd
from pathlib import Path
import re
from tqdm import tqdm
from math import ceil
import glob
import gc

tqdm.pandas()

In [3]:
dataset_version = '6.2'

answers_dir = Path(r"/kaggle/input/answers")
dataset_dir = Path(rf"/kaggle/input/reduced-data")

main_answers_file = answers_dir / "insiders.csv"

assert(answers_dir.is_dir())
assert(dataset_dir.is_dir())
assert(main_answers_file.is_file())

output_dir = Path(f'/kaggle/working/')
assert(output_dir.is_dir())

In [4]:
main_df = pd.read_csv(main_answers_file)
main_df = main_df[main_df['dataset'] == dataset_version]

malicious_users = main_df.user.unique()

In [5]:
main_df = pd.read_csv(main_answers_file)
main_df = main_df[main_df['dataset'] == dataset_version]

malicious_users = main_df.user.unique()

In [23]:
import pandas as pd
from pathlib import Path

# Define the dataset version (make sure this is defined in your code)
dataset_version = '6.1'  # or whatever version you're using

# Get list of files matching the pattern in answers_dir
insider_files = list(answers_dir.glob(f'**/r{dataset_version}*.csv'))

# List to store DataFrames
df_ls = []

# Check if any files were found
if not insider_files:
    print(f"No files found matching pattern: r{dataset_version}*.csv")
else:
    # Loop through the found files and load them into DataFrames
    for filename in insider_files:
        try:
            # Load the CSV file into a DataFrame, assuming 13 columns
            df = pd.read_csv(filename, names=list(range(13)))
            df_ls.append(df)
        except Exception as e:
            print(f"Error reading file {filename}: {e}")
    
    # Concatenate all DataFrames if any were loaded
    if df_ls:
        df = pd.concat(df_ls, axis=0, ignore_index=True)
        print(f"Concatenated DataFrame shape: {df.shape}")
    else:
        print("No DataFrames to concatenate.")


Concatenated DataFrame shape: (280, 13)


In [24]:
insider_files = list(answers_dir.glob(f'**/r{dataset_version}*.csv'))

df_ls = []

for filename in insider_files:
    df = pd.read_csv(filename, names=list(range(13)))
    df_ls.append(df)
    
df = pd.concat(df_ls, axis=0, ignore_index=True)

In [25]:
set(df[df[0] == 'http'][5].apply(lambda s: re.match('^https?://(www\.)?([0-9\-\w\.]+)?.+$', s).group(2)))

{'dropbox.com', 'wikileaks.org'}

In [28]:
device_cols = next(pd.read_csv('/kaggle/input/cert-insider-threat-detection-research/device.csv', chunksize=10)).columns
email_cols = next(pd.read_csv('/kaggle/input/cert-insider-threat-detection-research/email.csv', chunksize=10)).columns
file_cols = next(pd.read_csv('/kaggle/input/cert-insider-threat-detection-research/file.csv', chunksize=10)).columns
http_cols = next(pd.read_csv('/kaggle/input/reduced-data/http.csv', chunksize=10)).columns
logon_cols = next(pd.read_csv('/kaggle/input/cert-insider-threat-detection-research/logon.csv', chunksize=10)).columns

logon

In [30]:
df = pd.read_csv('/kaggle/input/cert-insider-threat-detection-research/logon.csv')

In [31]:
df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

In [32]:
df['day'] = df['date'].dt.floor('D')

# replaces df.groupby(['user', 'day']).pc.agg(pd.Series.mode)
# it much more complicated but much, much faster
# credit to: https://stackoverflow.com/a/57179083

# result is exactly same as "df.groupby('user').pc.agg(lambda x:x.value_counts().index[0])" though

most_common_pc = df\
    .groupby(['user', 'day', 'pc'])\
    .size()\
    .to_frame('count')\
    .reset_index()\
    .sort_values('count', ascending=False)\
    .drop_duplicates(subset=['user', 'day'])\
    .drop(columns=['count'])\
    .sort_values(['user', 'day'])\
    .groupby('user')\
    .pc\
    .agg(pd.Series.mode)\
    .rename('most_common_pc')
most_common_pc

user
AAB0162    PC-6599
AAB0398    PC-8719
AAC0610    PC-1834
AAC0668    PC-9677
AAC3270    PC-5225
            ...   
ZVB2656    PC-6247
ZVS1637    PC-6157
ZWS3625    PC-0695
ZXM3086    PC-8526
ZZO2997    PC-3120
Name: most_common_pc, Length: 4000, dtype: object

In [33]:
df = df.merge(most_common_pc, left_on='user', right_on='user', )
df['is_usual_pc'] = df['most_common_pc'] == df['pc']

is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
df['is_work_time'] = is_work_time

df['subtype'] = df['activity']

In [34]:
df[['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(output_dir / 'logon_preprocessed.csv')


Device

In [38]:
df = pd.read_csv('/kaggle/input/cert-insider-threat-detection-research/device.csv')
df

,id,date,user,pc,file_tree,activity
0,{Z2Q8-K3AV28BE-9353JIRT},01/02/2010 07:17:18,SDH2394,PC-5849,R:\;R:\22B5gX4;R:\SDH2394,Connect
1,{C7F1-G7LE60RU-2483DAXS},01/02/2010 07:22:42,JKS2444,PC-6961,R:\;R:\JKS2444,Connect
2,{T9A4-D4RV69OF-1704NINW},01/02/2010 07:31:42,CBA1023,PC-1570,R:\;R:\42gY283;R:\48rr4y2;R:\59ntt61;R:\76xCQG...,Connect
3,{S8L0-O6QQ15NL-0636OYNV},01/02/2010 07:33:28,GNT0221,PC-6427,R:\;R:\GNT0221,Connect
4,{U0F1-R1FX27FM-6954TTVU},01/02/2010 07:33:55,JKS2444,PC-6961,NaN,Disconnect
...,...,...,...,...,...,...
1551823,{H8F7-C0ZV44EZ-8096IKGM},05/31/2011 23:02:16,WSA1765,PC-7733,NaN,Disconnect
1551824,{I8V0-G3ID86HT-3033NLGV},05/31/2011 23:31:40,ABD3426,PC-8657,R:\;R:\ABD3426,Connect
1551825,{T5I0-I7MC85ZR-6708PJZP},05/31/2011 23:36:13,ABD3426,PC-8657,NaN,Disconnect
1551826,{A2P9-L4MQ58YG-5271LLTI},05/31/2011 23:46:23,QAH0048,PC-6551,R:\;R:\363xgw8;R:\QAH0048,Connect


In [39]:
df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

df = df.merge(most_common_pc, left_on='user', right_on='user', )
df['is_usual_pc'] = df['most_common_pc'] == df['pc']

is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
df['is_work_time'] = is_work_time

df['subtype'] = df['activity']

In [40]:
df[['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(output_dir / f'{csv_name}_preprocessed.csv')


file

In [72]:
df = pd.read_csv('/kaggle/input/cert-insider-threat-detection-research/file.csv', usecols=['date', 'user', 'pc', 'filename'])
df

,date,user,pc,filename
0,01/02/2010 07:19:41,SDH2394,PC-5849,R:\60WBQE7S.doc
1,01/02/2010 07:21:30,SDH2394,PC-5849,R:\0VGILDW8.pdf
2,01/02/2010 07:22:11,SDH2394,PC-5849,R:\60WBQE7S.doc
3,01/02/2010 07:24:06,SDH2394,PC-5849,R:\22B5gX4\H8Y96RRE.doc
4,01/02/2010 07:24:45,SDH2394,PC-5849,R:\SDH2394\7XRCV2N5.pdf
...,...,...,...,...
2014878,05/31/2011 23:36:00,ABD3426,PC-8657,R:\ABD3426\ZLKK3YO7.doc
2014879,05/31/2011 23:47:51,QAH0048,PC-6551,R:\363xgw8\VZIJJ9K4.doc
2014880,05/31/2011 23:48:23,QAH0048,PC-6551,R:\363xgw8\VZIJJ9K4.doc
2014881,05/31/2011 23:49:13,QAH0048,PC-6551,R:\363xgw8\VZIJJ9K4.doc


In [73]:
df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

df = df.merge(most_common_pc, left_on='user', right_on='user', )
df['is_usual_pc'] = df['most_common_pc'] == df['pc']

is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
df['is_work_time'] = is_work_time

file_extensions = df.filename.str[-4:]
df['subtype'] = file_extensions

In [74]:
df

,date,user,pc,filename,most_common_pc,is_usual_pc,is_work_time,subtype
0,2010-01-02 07:19:41,SDH2394,PC-5849,R:\60WBQE7S.doc,PC-5849,True,False,.doc
1,2010-01-02 07:21:30,SDH2394,PC-5849,R:\0VGILDW8.pdf,PC-5849,True,False,.pdf
2,2010-01-02 07:22:11,SDH2394,PC-5849,R:\60WBQE7S.doc,PC-5849,True,False,.doc
3,2010-01-02 07:24:06,SDH2394,PC-5849,R:\22B5gX4\H8Y96RRE.doc,PC-5849,True,False,.doc
4,2010-01-02 07:24:45,SDH2394,PC-5849,R:\SDH2394\7XRCV2N5.pdf,PC-5849,True,False,.pdf
...,...,...,...,...,...,...,...,...
2014878,2011-05-31 23:36:00,ABD3426,PC-8657,R:\ABD3426\ZLKK3YO7.doc,PC-8657,True,False,.doc
2014879,2011-05-31 23:47:51,QAH0048,PC-6551,R:\363xgw8\VZIJJ9K4.doc,PC-2247,False,False,.doc
2014880,2011-05-31 23:48:23,QAH0048,PC-6551,R:\363xgw8\VZIJJ9K4.doc,PC-2247,False,False,.doc
2014881,2011-05-31 23:49:13,QAH0048,PC-6551,R:\363xgw8\VZIJJ9K4.doc,PC-2247,False,False,.doc


In [75]:
df[['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(output_dir / 'file_preprocessed.csv')

email

In [76]:
df = pd.read_csv('/kaggle/input/cert-insider-threat-detection-research/email.csv', usecols=['date', 'user', 'pc', 'to', 'cc', 'bcc', 'from'])
df = df.fillna('')
df

,date,user,pc,to,cc,bcc,from
0,01/02/2010 06:36:41,HDB1666,PC-6793,Louis.Bernard.Garza@dtaa.com,Emery.Ali.Holloway@dtaa.com,Hector.Donovan.Bray@dtaa.com,Hector.Donovan.Bray@dtaa.com
1,01/02/2010 06:40:02,HDB1666,PC-6793,Hector.Donovan.Bray@dtaa.com,,,Luke.Grant.Mcmahon@dtaa.com
2,01/02/2010 06:42:48,HDB1666,PC-6793,Quintessa.O.Farrell@harris.com,Hector.Donovan.Bray@dtaa.com,,Hector.Donovan.Bray@dtaa.com
3,01/02/2010 06:45:42,HDB1666,PC-6793,Hector.Donovan.Bray@dtaa.com,,,Travis.Ezra.Warner@dtaa.com
4,01/02/2010 06:47:07,HDB1666,PC-6793,Hector.Donovan.Bray@dtaa.com,,,Kenyon.William.Delacruz@dtaa.com
...,...,...,...,...,...,...,...
10994952,05/31/2011 19:58:30,GLN2107,PC-6169,Noel_Gay@verizon.net,,,Emma.G.Potter@hotmail.com
10994953,05/31/2011 20:01:00,UMB0055,PC-1021,Teegan.F.Leonard@northropgrumman.com,Ursa.Melinda.Bentley@dtaa.com,,Ursa.Melinda.Bentley@dtaa.com
10994954,05/31/2011 20:01:42,GLN2107,PC-6169,Gay.Lunea.Noel@dtaa.com,,,Emma.Galena.Potter@dtaa.com
10994955,05/31/2011 20:05:04,GLN2107,PC-6169,Michelle.Britanney.Peters@dtaa.com;Emma.Galena...,,,Gay.Lunea.Noel@dtaa.com


In [77]:
from_domain = df['from'].str.extract('^.+@(.+$)')[0]
is_external_from = from_domain == 'dtaa.com'

In [78]:
# this lines takes ~10 mins
# to_concated = df[['to', 'cc', 'bcc']].agg(';'.join, axis=1)

# slighly slower but there is nice progress bar
to_concated = df[['to', 'cc', 'bcc']].progress_apply(lambda x: ';'.join([x.to, x.cc, x.bcc]), axis=1)

100%|██████████| 10994957/10994957 [08:04<00:00, 22684.00it/s]


In [79]:
# yes, it's horrible but this part is somewhat fast compared to the join part

is_external_to = to_concated.progress_apply(
    lambda x: any([re.match('^.+@(.+$)', e).group(1) != 'dtaa.com' for e in x.split(';') if e != ''])
    )

100%|██████████| 10994957/10994957 [01:43<00:00, 105830.81it/s]


In [80]:
is_external = is_external_to | is_external_to

In [81]:
df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

df = df.merge(most_common_pc, left_on='user', right_on='user', )
df['is_usual_pc'] = df['most_common_pc'] == df['pc']

is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
df['is_work_time'] = is_work_time

df['subtype'] = is_external

In [82]:
df[['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(output_dir / 'email_preprocessed.csv')

http

In [52]:
job_hunting_websites = [
    'careerbuilder.com',
    'craiglist.org',
    'indeed.com',
    'job-hunt.org',
    'jobhuntersbible.com',
    'linkedin.com',
    'monster.com',
    'simplyhired.com',
]

hacktivist_websites = [
    'actualkeylogger.com',
    'best-spy-soft.com',
    'dailykeylogger.com',
    'keylogpc.com',
    'refog.com',
    'relytec.com',
    'softactivity.com',
    'spectorsoft.com',
    'webwatchernow.com',
    'wellresearchedreviews.com',
    'wikileaks.org'
]

filesharing_websites = [
    '4shared.com'
    'dropbox.com',
    'fileserve.com',
    'filefreak.com',
    'filestube.com',
    'megaupload.com',
    'thepiratebay.org'
]

In [53]:
with open(dataset_dir / 'http.csv') as f:
    for count, l in tqdm(enumerate(f)):
        pass
count

1000it [00:00, 76000.29it/s]


999

In [54]:
CHUNK_SIZE = 500000

df_iter = pd.read_csv(dataset_dir / 'http.csv', chunksize=CHUNK_SIZE, usecols=['date', 'user', 'pc', 'url'])
# (output_dir / 'http_preprocessed.csv').unlink()
first_it = True
mode = 'w'

for http_df in tqdm(df_iter, total=ceil(count / CHUNK_SIZE)):
    http_df['date'] = pd.to_datetime(http_df.date, format='%m/%d/%Y %H:%M:%S')

    site_names = http_df['url'].apply(lambda s: re.match('^https?://(www)?([0-9\-\w\.]+)?.+$', s).group(2))
    http_df['site_name'] = site_names
    
    http_df['subtype'] = 0
    http_df.loc[site_names.isin(job_hunting_websites), 'subtype'] = 1
    http_df.loc[site_names.isin(hacktivist_websites), 'subtype'] = 2
    http_df.loc[site_names.isin(filesharing_websites), 'subtype'] = 3
    
    http_df = http_df.merge(most_common_pc, left_on='user', right_on='user', )
    http_df['is_usual_pc'] = http_df['most_common_pc'] == http_df['pc']

    is_work_time = (8 <= http_df.date.dt.hour) & (http_df.date.dt.hour < 17)
    http_df['is_work_time'] = is_work_time
    
    http_df.to_csv(output_dir / 'http_preprocessed.csv', header=first_it, index=False,
                   mode=mode, columns=['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype', 'site_name'])
    first_it = False
    mode = 'a'

100%|██████████| 1/1 [00:00<00:00, 25.97it/s]


http

In [57]:
from pathlib import Path

# Correctly defining LDAP_dir as a Path object
LDAP_dir = Path('/kaggle/input/cert-insider-threat-detection-research/LDAP')

# Check if the directory exists
assert LDAP_dir.is_dir(), f"The directory {LDAP_dir} does not exist."

# Define the LDAP columns you're interested in
ldap_columns = ['role', 'business_unit', 'functional_unit', 'department', 'team']

print(f"The directory {LDAP_dir} exists.")


The directory /kaggle/input/cert-insider-threat-detection-research/LDAP exists.


In [58]:
# List files in the LDAP directory
ldap_files = list(LDAP_dir.glob('*'))  # Lists all files in the LDAP directory

# Display the files found in the LDAP directory
for file in ldap_files:
    print(file)


/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-08.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2011-03.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-10.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-02.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-03.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-09.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2011-04.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2009-12.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-06.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-07.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-05.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-11.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-04.csv
/kaggle/input/cert-insider-threat-detection-research/LDAP/2010-12.csv
/kaggle/input/cert-i

In [60]:
import pandas as pd

# Initialize a dictionary to hold DataFrames
ldap_dataframes = {}

# Loop through each CSV file in the LDAP directory
for file in ldap_files:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file, usecols=ldap_columns)
    
    # Store the DataFrame in the dictionary with the file name as the key
    ldap_dataframes[file.name] = df
    print(f"Loaded {file.name} with shape {df.shape}")

# Example: Access a specific DataFrame
# df_specific = ldap_dataframes['your_file_name.csv']


Loaded 2010-08.csv with shape (3853, 5)
Loaded 2011-03.csv with shape (3674, 5)
Loaded 2010-10.csv with shape (3803, 5)
Loaded 2010-02.csv with shape (3977, 5)
Loaded 2010-03.csv with shape (3952, 5)
Loaded 2010-09.csv with shape (3833, 5)
Loaded 2011-04.csv with shape (3654, 5)
Loaded 2009-12.csv with shape (4000, 5)
Loaded 2010-06.csv with shape (3894, 5)
Loaded 2010-07.csv with shape (3870, 5)
Loaded 2010-05.csv with shape (3911, 5)
Loaded 2010-11.csv with shape (3773, 5)
Loaded 2010-04.csv with shape (3930, 5)
Loaded 2010-12.csv with shape (3744, 5)
Loaded 2011-01.csv with shape (3719, 5)
Loaded 2011-02.csv with shape (3697, 5)
Loaded 2011-05.csv with shape (3639, 5)
Loaded 2010-01.csv with shape (4000, 5)


In [61]:
# Initialize a list to hold individual DataFrames
df_list = []

# Loop through each CSV file in the LDAP directory
for file in ldap_files:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file, usecols=ldap_columns)
    df_list.append(df)  # Add the DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
df_combined = pd.concat(df_list, ignore_index=True)
print("Combined DataFrame shape:", df_combined.shape)

# Display the first few rows of the combined DataFrame
print(df_combined.head())


Combined DataFrame shape: (68923, 5)
                   role  business_unit  \
0               ITAdmin              1   
1  ProductionLineWorker              1   
2  ProductionLineWorker              1   
3  ProductionLineWorker              1   
4    MechanicalEngineer              1   

                                  functional_unit       department  \
0                               1 - Adminstration     5 - Security   
1                    5 - Manufacturing_Commercial     3 - Assembly   
2                    6 - Manufacturing_Government     3 - Assembly   
3                    6 - Manufacturing_Government     3 - Assembly   
4  3 - ResearchAndEngineering_Government_Domestic  4 - Engineering   

                      team  
0   8 - ElectronicSecurity  
1         4 - AssemblyDept  
2        14 - AssemblyDept  
3         8 - AssemblyDept  
4  16 - TestAndEvalualtion  


In [69]:
import pandas as pd
import gc
from pathlib import Path

# Define the directory where your LDAP files are located
LDAP_dir = Path('/kaggle/input/cert-insider-threat-detection-research/LDAP')

# Specify the columns you want to retain
ldap_columns = ['role', 'business_unit', 'functional_unit', 'department', 'team']

# List to store DataFrames
ldap_df_list = []

# Loop through each CSV file in the directory
for file_path in LDAP_dir.glob('*.csv'):
    # Extract year and month from the filename (assuming it's formatted as 'YYYY-MM')
    year, month = file_path.stem.split('-')
    
    # Read the CSV file into a DataFrame
    ldap_df = pd.read_csv(file_path)
    
    # Ensure only the desired columns and add 'user_id'
    ldap_df = ldap_df[ldap_columns + ['user_id']]
    
    # Convert year and month to integers and add to DataFrame
    ldap_df['year'] = int(year)
    ldap_df['month'] = int(month)
    
    # Append the DataFrame to the list
    ldap_df_list.append(ldap_df)

# Concatenate all DataFrames into a single DataFrame
ldap_df = pd.concat(ldap_df_list, ignore_index=True)

# Optional: Clean up memory
del ldap_df_list
gc.collect()

# Display the shape and the first few rows of the concatenated DataFrame
print(f"Combined DataFrame shape: {ldap_df.shape}")
print(ldap_df.head())


Combined DataFrame shape: (68923, 8)
                   role  business_unit  \
0               ITAdmin              1   
1  ProductionLineWorker              1   
2  ProductionLineWorker              1   
3  ProductionLineWorker              1   
4    MechanicalEngineer              1   

                                  functional_unit       department  \
0                               1 - Adminstration     5 - Security   
1                    5 - Manufacturing_Commercial     3 - Assembly   
2                    6 - Manufacturing_Government     3 - Assembly   
3                    6 - Manufacturing_Government     3 - Assembly   
4  3 - ResearchAndEngineering_Government_Domestic  4 - Engineering   

                      team  user_id  year  month  
0   8 - ElectronicSecurity  NFP2441  2010      8  
1         4 - AssemblyDept  ADR1517  2010      8  
2        14 - AssemblyDept  MWB4000  2010      8  
3         8 - AssemblyDept  MLS2856  2010      8  
4  16 - TestAndEvalualtion  BTR202

In [70]:
users = ldap_df.user_id.unique()
for user in users:
    user_df = ldap_df[ldap_df['user_id'] == user]
    for col in ldap_columns:
        assert user_df[col].unique().shape[0] == 1

This means users in CERT 4.2 do not change their units, departments, etc.

In [71]:
ldap_df = pd.read_csv(LDAP_dir / '2009-12.csv')
for col in ldap_columns:
    ldap_df[col] = ldap_df[col].astype('category')
ldap_df = ldap_df[['user_id'] + ldap_columns]

ldap_df.to_csv(output_dir / 'LDAP.csv', index=False)